# **Roadmap**

1. Install the TensorFlow Object Detection API
2. Setup folder structure
3. Generate the TFRecord files required for training
4. Edit the model pipeline config file and download the pre-trained model checkpoint
5. Train and evaluate the model

⚠️ This notebook is meant to be run in Google Colab for training in order to use GPU capacity.

In [ ]:
# Check GPU setup

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# Check RAM setup

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# **Import libraries**

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.8.2


# 1. Create customTF2, training and data folders in your google drive

Create a folder named ***customTF2***.

Create another folder named ***training*** inside the ***customTF2*** folder
(***training*** folder is where the checkpoints will be saved during training)

Create another folder named ***data*** inside the ***customTF2*** folder.

In [ ]:
## Test